<a href="https://colab.research.google.com/github/AdrieonK/Bioinformatics-Data-Science/blob/main/heart_failure_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Goal: Build a Binary Classification Model to predict patient survival from all or a select set of features**

In [ ]:
#imports

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

##**Dataset**

In [ ]:
#unzip dataset file

!unzip /content/heart+failure+clinical+records.zip

Archive:  /content/heart+failure+clinical+records.zip
 extracting: heart_failure_clinical_records_dataset.csv  


In [ ]:
#open file

heart_failure_data = pd.read_csv('/content/heart_failure_clinical_records_dataset.csv')

heart_failure_data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


##**Process Data**

In [ ]:
heart_failure_data.columns

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT'],
      dtype='object')

In [ ]:
print(heart_failure_data.dtypes)

age                         float64
anaemia                       int64
creatinine_phosphokinase      int64
diabetes                      int64
ejection_fraction             int64
high_blood_pressure           int64
platelets                   float64
serum_creatinine            float64
serum_sodium                  int64
sex                           int64
smoking                       int64
time                          int64
DEATH_EVENT                   int64
dtype: object


In [ ]:
heart_failure_data.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [ ]:
heart_failure_data.isnull()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,False,False,False,False,False,False,False,False,False,False,False,False,False
295,False,False,False,False,False,False,False,False,False,False,False,False,False
296,False,False,False,False,False,False,False,False,False,False,False,False,False
297,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
scaler = StandardScaler()

continuous_data_columns = ['age', 'platelets', 'serum_creatinine']

heart_failure_data[continuous_data_columns] = scaler.fit_transform(heart_failure_data[continuous_data_columns])


In [ ]:
heart_failure_data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,1.192945,0,582,0,20,1,1.681648e-02,0.490057,130,1,0,4,1
1,-0.491279,0,7861,0,38,0,7.535660e-09,-0.284552,136,1,0,6,1
2,0.350833,0,146,0,20,0,-1.038073e+00,-0.090900,129,1,1,7,1
3,-0.912335,1,111,0,20,0,-5.464741e-01,0.490057,137,1,0,7,1
4,0.350833,1,160,1,20,0,6.517986e-01,1.264666,116,0,0,8,1


##**Train/Test/Split**

In [ ]:
heart_failure_data_features = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']

X = heart_failure_data[heart_failure_data_features]

y = heart_failure_data.DEATH_EVENT

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .2, random_state = 1)

In [ ]:
classifier = RandomForestClassifier(random_state = 1, )

classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_val)

print("Accuracy Score", accuracy_score(y_val, y_pred))
print("Classification Report", classification_report(y_val, y_pred))

Accuracy Score 0.9333333333333333
Classification Report               precision    recall  f1-score   support

           0       0.94      0.98      0.96        46
           1       0.92      0.79      0.85        14

    accuracy                           0.93        60
   macro avg       0.93      0.88      0.90        60
weighted avg       0.93      0.93      0.93        60



In [ ]:
#imports

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

##**Dataset**

In [ ]:
#unzip dataset file

!unzip /content/heart+failure+clinical+records.zip

Archive:  /content/heart+failure+clinical+records.zip
 extracting: heart_failure_clinical_records_dataset.csv  


In [ ]:
#open file

heart_failure_data = pd.read_csv('/content/heart_failure_clinical_records_dataset.csv')

heart_failure_data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


##**Process Data**

In [ ]:
heart_failure_data.columns

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT'],
      dtype='object')

In [ ]:
print(heart_failure_data.dtypes)

age                         float64
anaemia                       int64
creatinine_phosphokinase      int64
diabetes                      int64
ejection_fraction             int64
high_blood_pressure           int64
platelets                   float64
serum_creatinine            float64
serum_sodium                  int64
sex                           int64
smoking                       int64
time                          int64
DEATH_EVENT                   int64
dtype: object


In [ ]:
heart_failure_data.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [ ]:
heart_failure_data.isnull()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,False,False,False,False,False,False,False,False,False,False,False,False,False
295,False,False,False,False,False,False,False,False,False,False,False,False,False
296,False,False,False,False,False,False,False,False,False,False,False,False,False
297,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
scaler = StandardScaler()

continuous_data_columns = ['age', 'platelets', 'serum_creatinine']

heart_failure_data[continuous_data_columns] = scaler.fit_transform(heart_failure_data[continuous_data_columns])


In [ ]:
heart_failure_data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,1.192945,0,582,0,20,1,1.681648e-02,0.490057,130,1,0,4,1
1,-0.491279,0,7861,0,38,0,7.535660e-09,-0.284552,136,1,0,6,1
2,0.350833,0,146,0,20,0,-1.038073e+00,-0.090900,129,1,1,7,1
3,-0.912335,1,111,0,20,0,-5.464741e-01,0.490057,137,1,0,7,1
4,0.350833,1,160,1,20,0,6.517986e-01,1.264666,116,0,0,8,1


##**Train/Test/Split**

In [ ]:
heart_failure_data_features = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking']

X = heart_failure_data[heart_failure_data_features]

y = heart_failure_data.DEATH_EVENT

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .2, random_state = 1)

In [ ]:
classifier = RandomForestClassifier(random_state = 1, class_weight = 'balanced')

classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_val)

print("Accuracy Score", accuracy_score(y_val, y_pred))
print("Classification Report", classification_report(y_val, y_pred))

Accuracy Score 0.8
Classification Report               precision    recall  f1-score   support

           0       0.85      0.89      0.87        46
           1       0.58      0.50      0.54        14

    accuracy                           0.80        60
   macro avg       0.72      0.70      0.71        60
weighted avg       0.79      0.80      0.79        60



In [ ]:
feature_importances = classifier.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)
print(importance_df)

                     Feature  Importance
7           serum_creatinine    0.205380
4          ejection_fraction    0.159071
2   creatinine_phosphokinase    0.143876
6                  platelets    0.131564
0                        age    0.128422
8               serum_sodium    0.122773
9                        sex    0.023403
1                    anaemia    0.022811
3                   diabetes    0.022386
5        high_blood_pressure    0.021105
10                   smoking    0.019209


In [ ]:
heart_failure_data_features = [
       'ejection_fraction',
       'serum_creatinine']

X = heart_failure_data[heart_failure_data_features]

y = heart_failure_data.DEATH_EVENT

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .3, random_state = 1)

In [ ]:
classifier = RandomForestClassifier(random_state = 1)

classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_val)

print("Accuracy Score", accuracy_score(y_val, y_pred))
print("Classification Report", classification_report(y_val, y_pred))

Accuracy Score 0.7666666666666667
Classification Report               precision    recall  f1-score   support

           0       0.83      0.84      0.84        64
           1       0.60      0.58      0.59        26

    accuracy                           0.77        90
   macro avg       0.72      0.71      0.71        90
weighted avg       0.76      0.77      0.77        90



In [ ]:
feature_importances = classifier.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)
print(importance_df)

             Feature  Importance
1   serum_creatinine    0.588897
0  ejection_fraction    0.411103
